In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from google.colab import files
from google.colab import drive

In [ ]:
!pip3 install pytorch_lightning
import pytorch_lightning as pl

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 705 kB 26.2 MB/s 
     |████████████████████████████████| 419 kB 52.1 MB/s 
     |████████████████████████████████| 5.9 MB 48.4 MB/s 
     |████████████████████████████████| 596 kB 61.6 MB/s 
     |████████████████████████████████| 141 kB 54.7 MB/s 
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires tensorboard<2.9,>=2.8, but you have tensorboard 2.10.0 which is in

In [ ]:
drive.mount('/drive')

In [ ]:
import re
pattern = "^[A-Za-z0-9_-]*$"
tweets = tweets[tweets.apply(lambda x: 
          bool(re.match(pattern, x['hashtags'])), axis=1)]

In [ ]:
# # pd.set_option('display.max_rows', None)
# tweets = pd.read_csv('/drive/My Drive/cs297/FeiPan_TwitterRecommender_CS297/code/NCF/data/processed_hashtags.csv')
# # tweets = pd.read_csv('/drive/My Drive/cs297/FeiPan_TwitterRecommender_CS297/code/NCF/data/downloaded_hashtags_extend.csv')
# # TO-DO: drop non-english hashtags (1264114113925402624, 177400353, 1512718628223074306)
# print(tweets.shape)
# # tweets.loc[tweets['hashtags'].isalpha()]\
# # df = tweets[tweets.apply(lambda x: x['hashtags'].isalnum(), axis=1)]
# df = tweets[tweets.apply(lambda x: bool(re.match(pattern, x['hashtags'])), axis=1)]

# # df = tweets[tweets.apply(lambda x: detector.detect(x['hashtags']).lang == 'en', axis=1)]
    
# df2 = tweets.loc[set(tweets.index) - set(df.index.get_level_values(0))]
# tweets.reset_index()
# print(tweets.shape)
# # print(tweets)
# print(df2.shape)
# # print(df2)
# df2.dtypes
# # df2.iloc[:, 1:2]

In [ ]:
# process hashtags: drop duplicate and keep last, deop non-english hashtags
df = pd.read_csv('/drive/My Drive/cs297/FeiPan_TwitterRecommender_CS297/code/NCF/data/downloaded_hashtags_extend.csv')
print(df.shape)
tweets = df.drop_duplicates(subset=['author', 'hashtags'], keep='last')
print(tweets.shape)
tweets = tweets[tweets.apply(lambda x: bool(re.match(pattern, x['hashtags'])), axis=1)]
df1 = tweets[tweets.apply(lambda x: bool(re.match(pattern, x['hashtags'])), axis=1)]
df2 = tweets.loc[set(tweets.index) - set(df1.index.get_level_values(0))]
print(df1.shape)
print(df2.shape)
print(tweets.shape)
# # print(tweets['hashtags']) 

In [ ]:
counts_user_freq = tweets.groupby("author")["author"].transform(len)
mask = counts_user_freq >= 50
tweets = tweets[mask]
print(tweets.shape)

In [ ]:
pd.set_option('display.max_rows', None)
print(df2['hashtags'])

In [ ]:
# Lable Encoding users and hashtags
all_users = tweets['author'].unique()
all_hashtags = tweets['hashtags'].unique()
print(len(all_users), all_users)
print(len(all_hashtags), all_hashtags)

users_encoding = {k: v for v, k in enumerate(all_users)}
hashtags_encoding = {k: v for v, k in enumerate(all_hashtags)}
print(users_encoding)

In [ ]:
def mapping_users(row):
    return users_encoding[row['author']]


def mapping_hashtags(row):
    return hashtags_encoding[row['hashtags']]


tweets['author_encoded'] = tweets.apply(mapping_users, axis=1)
tweets['hashtag_encoded'] = tweets.apply(mapping_hashtags, axis=1)
tweets.shape

In [ ]:
# leave-one-out train-test split
# pd.set_option('display.max_row', None)
# tweets['date'] = pd.to_datetime(tweets['date'])
# tweets['latest'] = tweets.groupby(['author_encoded'])['date'].rank(method='first', ascending=False)
# print(tweets)

# tweets_train = tweets[tweets['latest'] != 1]
# tweets_test = tweets[tweets['latest'] == 1]



In [ ]:
tweets['date'] = pd.to_datetime(tweets['date'])
tweets['latest'] = tweets.groupby(['author_encoded'])['date'].rank(method='first', ascending=False)
print(tweets.shape)
# tweets_train = tweets.groupby('author_encoded').apply(lambda x: x.sample(frac=0.8))

tweets_test = tweets.groupby('author_encoded').head(10)
print(tweets_test.shape)
# print(tweets_test)
# print(len(tweets_test['author'].unique())
tweets_train = tweets_test.merge(tweets, how = 'outer' ,indicator=True).loc[lambda x : x['_merge']=='right_only']
tweets_train = tweets_train.drop('_merge', axis=1)
print(tweets_train.shape)

In [ ]:
tweets_test.shape

In [ ]:
tweets_train

In [ ]:
tweets_test

In [ ]:
# print(tweets.groupby(['author_encoded']).size())
# tweets_train.groupby(['author_encoded']).size()
tweets_test.groupby(['author_encoded']).size()
print(tweets_test.groupby(['author_encoded']).size())

In [ ]:
# drop unnecessary column
tweets_train = tweets_train[['author_encoded', 'hashtag_encoded']]
tweets_train.loc[:, 'label'] = 1
tweets_test = tweets_test[['author_encoded', 'hashtag_encoded']]
tweets_test.loc[:, 'label'] = 1
tweets_train = tweets_train.reset_index(drop=True)
tweets_test = tweets_test.reset_index(drop=True)
# print(tweets_train)
# print(tweets_test)

In [ ]:
# add negative labels
users, hashtags, labels = [], [], []
user_hashtag_set = set(zip(tweets_train['author_encoded'], tweets_train['hashtag_encoded']))
all_users_encoded = tweets_test['author_encoded'].unique().tolist()
print(len(all_users_encoded))
# print([1 2 3 4] + [5 6]])
all_hashtags_encoded = tweets['hashtag_encoded'].unique()
print(len(all_hashtags_encoded))

ratio = 4   # negative / positive
for (u, h) in user_hashtag_set:
    users.append(u)
    hashtags.append(h)
    labels.append(1)
    for _ in range(ratio):
        neg_hashtag = np.random.choice(all_hashtags_encoded)
        while (u, neg_hashtag) in user_hashtag_set:
            neg_hashtag = np.random.choice(all_hashtags_encoded)
        users.append(u)
        hashtags.append(neg_hashtag)
        labels.append(0)

In [ ]:
# print(users)
# print(hashtags)
# print(labels)
# print(len(labels))

In [ ]:
print(tweets)

In [ ]:
class HashtagsTrainDataset(Dataset):
    def __init__(self, tweets, all_hashtagIds):
        self.users, self.items, self.labels = self.get_dataset(tweets, all_hashtagIds)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, tweets, all_hashtagIds):
        users, items, labels = [], [], []
        user_item_set = set(zip(tweets['author_encoded'], tweets['hashtag_encoded']))

        num_negatives = 4
        for u, i in user_item_set:
            users.append(u)
            items.append(i)
            labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_hashtagIds)
                while (u, negative_item) in user_item_set:
                    negative_item = np.random.choice(all_hashtagIds)
                users.append(u)
                items.append(negative_item)
                labels.append(0)
        # print("~~~ users ", len(users), type(users))
        # print(users)
        # print("~~~ items", len(items))
        # print(items)
        # print("~~~labels", len(labels))
        # print(labels)
        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)
        #return torch.tensor([[u] for u in users]), torch.tensor([[i] for i in items]), torch.tensor([[l] for l in labels])

In [ ]:
from numpy.ma.core import shape
# Neural Collaborative Filtering
class NCF(pl.LightningModule):
    def __init__(self, num_users, num_items, tweets, all_hashtagIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.tweets = tweets
        self.all_hashtagIds = all_hashtagIds

    def forward(self, user_input, item_input):
        # print("~~~ in forward", user_input.size())
        # print(torch.LongTensor([[0,2,0,5]]).size())
        # one-hot to 8d embedded
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)
        # contact
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # to dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # output
        pred = nn.Sigmoid()(self.output(vector))

        return pred

    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        # print("~~~ in training step", user_input.size(), item_input.size(), labels.size())
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(HashtagsTrainDataset(self.tweets, self.all_hashtagIds),
                          batch_size=512, num_workers=2)

In [ ]:

num_users = tweets['author_encoded'].max() + 1
print(tweets['author_encoded'])
num_items = tweets['hashtag_encoded'].max() + 1
print(tweets['hashtag_encoded'])
all_hashtags_encoded = tweets['hashtag_encoded'].unique()

model = NCF(num_users, num_items, tweets_train, all_hashtags_encoded)
print(tweets_train)

# trainer = pl.Trainer(max_epochs=5, gpus=None, reload_dataloaders_every_n_epochs=True,
#                      progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer = pl.Trainer(max_epochs=5, gpus=None, reload_dataloaders_every_n_epochs=True,
                     enable_progress_bar=True, logger=False, enable_checkpointing=False)
trainer.fit(model)

In [ ]:
# User-item pairs for testing
test_user_item_set = set(zip(tweets_test['author_encoded'], tweets_test['hashtag_encoded']))
print(test_user_item_set)

{(109, 365), (124, 17856), (35, 7226), (7, 174), (10, 1835), (122, 12610), (102, 14649), (42, 7706), (106, 15999), (111, 123), (135, 13940), (6, 509), (73, 12244), (16, 2275), (39, 7430), (47, 8990), (150, 12233), (145, 1959), (59, 10169), (78, 6438), (82, 4690), (106, 619), (97, 5836), (73, 7471), (96, 14329), (72, 11122), (121, 17777), (93, 14152), (121, 1506), (125, 219), (56, 9896), (101, 1773), (0, 5), (26, 4423), (103, 8055), (59, 10167), (144, 19941), (63, 10611), (151, 3976), (6, 514), (74, 12270), (138, 18074), (77, 12596), (96, 14327), (127, 16545), (61, 10396), (45, 1540), (138, 13688), (91, 10654), (149, 14578), (100, 1803), (44, 2624), (87, 13675), (124, 11908), (2, 267), (61, 10389), (139, 3215), (113, 1476), (69, 7843), (20, 3127), (26, 4425), (107, 16032), (136, 160), (37, 7342), (143, 2917), (15, 2198), (98, 3314), (23, 3324), (25, 4130), (28, 4582), (48, 511), (5, 442), (33, 5513), (83, 822), (14, 1506), (1, 149), (21, 3188), (54, 9410), (25, 1342), (31, 4776), (44, 1

In [ ]:
# Dict of all items that are interacted with by each user
user_interacted_items = tweets.groupby('author_encoded')['hashtag_encoded'].apply(list).to_dict()
user_interacted_items

In [ ]:
def compute_apk(selected_items, predict_hashtags_encoded, k):
    hits = 0
    score = 0.0

    for i, num in enumerate(predict_hashtags_encoded) :
        if num in selected_items:
            hits += 1
            score += hits / (i + 1)

    return score / k

In [ ]:
def compute_ark(selected_items, predict_hashtags_encoded, k):
    tp = 0
    fn = 0
    score = 0.0

    for i, num in enumerate(predict_hashtags_encoded) :
        if num in selected_items:
          if i < len(selected_items):
            tp += 1
          else:
            fn += 1
          score += tp / (tp + fn)

    return score / k

In [ ]:
def compute_rrk(selected_items, predict_hashtags_encoded):
    for i, num in enumerate(predict_hashtags_encoded) :
        if num in selected_items:
            return 1 / (i + 1)

    return 0

In [ ]:
# def compute_ark(selected_items, predict_hashtags_encoded, k):
#     hits = 0
#     score = 0.0

#     for i, num in enumerate(predict_hashtags_encoded) :
#         if num in selected_items:
#             hits += 1
#             score += hits / k

#     return score / k

In [ ]:
hits = [0 for _ in range(len(all_users_encoded))]

test_sizes = tweets_test.groupby(['author_encoded']).size().tolist()
count = 0
total_precisions_score = 0.0
total_recall_score = 0.0
total_reciprocal_rank = 0.0
for u in all_users_encoded:
    interacted_items = user_interacted_items[u]
    not_interacted_items = set(all_hashtags_encoded) - set(interacted_items)
    len_test_data = test_sizes[u]
    # if len_test_data < 10 :
    #     selected_not_interacted = list(np.random.choice(list(not_interacted_items), 100 - len_test_data))
    # else :
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 5))
    
    selected_items = tweets_test.loc[tweets_test['author_encoded'] == u]['hashtag_encoded'].head(5).tolist()
    test_items = selected_items + selected_not_interacted
    # print(len(test_items), len(selected_items), len(selected_not_interacted))

    predicted_labels = np.squeeze(model(torch.tensor([u]*10), 
                                        torch.tensor(test_items)).detach().numpy())
    print(predicted_labels)

    top10_items = [test_items[i] for i in np.argsort(predicted_labels)[::-1][0:10].tolist()]
    predict_hashtags = [all_hashtags[top10_items[x]] for x in range(10)]
    predict_hashtags_encoded = [top10_items[x] for x in range(10)]

    matched = ""
    for i in selected_items:
        if i in top10_items:
            matched = all_hashtags[i]
            hits[u] = 1
            count += 1
            break
    print('~~~~~~~~~~~~~~ user:', u,  matched)
    print(selected_items)
    print(predict_hashtags_encoded)
    average_precision = compute_apk(selected_items, predict_hashtags_encoded,5)
    print('apk',average_precision)
    total_precisions_score += average_precision

    average_recall = compute_ark(selected_items, predict_hashtags_encoded,5)
    print('ark',average_recall)
    total_recall_score += average_recall

    rrk = compute_rrk(selected_items, predict_hashtags_encoded)
    print('rrk: ', rrk) 
    total_reciprocal_rank += rrk

    print([all_hashtags[selected_items[x]] for x in range(len(selected_items))])
    print(predict_hashtags)

mapk = total_precisions_score / len(all_users_encoded)
mark = total_recall_score / len(all_users_encoded)
mrrk = total_reciprocal_rank / len(all_users_encoded)
print('mapk:', mapk, 'mark:', mark, 'mrrk:', mrrk)
print(hits)
print(tweets.groupby(['author_encoded']).size().tolist())
print("The Hit Ratio @ 10 is {:.2f}".format(np.average(hits)))
print(count, "out of", len(all_users_encoded))    


[0.11706588 0.2139345  0.20405766 0.12142473 0.13898501 0.17233497
 0.15711081 0.24376704 0.10989365 0.15342014]
~~~~~~~~~~~~~~ user: 0 awesomepeopleontwitter
[0, 1, 2, 3, 4]
[6489, 1, 2, 17588, 21309, 17537, 4, 3, 0, 3929]
apk 0.53015873015873
ark 0.7133333333333333
rrk:  0.5
['awesomepeopleontwitter', 'npr', 'obgyntwitter', 'bodybreak', 'immunizeunder5s']
['endtheconvoys', 'npr', 'obgyntwitter', 'shadowban', 'financialfreedom', 'christofascist', 'immunizeunder5s', 'bodybreak', 'awesomepeopleontwitter', 'dumbass']
[0.16715403 0.20778628 0.2110826  0.1708441  0.14281665 0.11381724
 0.17632438 0.15350671 0.13833483 0.16948757]
~~~~~~~~~~~~~~ user: 1 yoga
[141, 142, 143, 144, 145]
[143, 142, 17534, 144, 5992, 141, 12859, 145, 14582, 4347]
apk 0.8083333333333332
ark 0.8699999999999999
rrk:  1.0
['yoga', 'buffalobayoupark', 'onmywalktoday', 'bragg', 'braggiscorrupt']
['onmywalktoday', 'buffalobayoupark', 'one2day', 'bragg', 'ovrsite', 'yoga', 'harrychella', 'braggiscorrupt', 'dontsayblack'